# Get RGTDB Entries for One User

https://rgtdb.com/user/72763

In [1]:
import pandas as pd
from datetime import datetime as dt
import datetime

dfs = pd.read_html('https://rgtdb.com/user/72763')
df = dfs[1]

## Convert date to datetime, attempt to guess the year

This is going to have troubles around New Year

In [2]:
event_year = str(dt.today().year)
year_subtract = 0
last_month = 13

for index, row in df.iterrows():
    event_month = int(row['Date'][:2])

    if event_month > last_month:
        year_subtract += 1
        event_year = str(dt.today().year - year_subtract)
        print(event_year + '-' + str(event_month))

    df.loc[index, 'Year_Date'] = event_year + '-' + row['Date']

    last_month = event_month
    
df['Year_Date'] = pd.to_datetime(df['Year_Date'], utc=True)
df.set_index('Year_Date', inplace=True)
df

2020-12


,Date,Name
Year_Date,,
2021-04-10 11:00:00+00:00,04-10 11:00,WKG's SDW. Stage 5
2021-04-09 05:00:00+00:00,04-09 05:00,The Endurance Ride
2021-04-07 04:00:00+00:00,04-07 04:00,The Recovery Ride
2021-04-05 07:00:00+00:00,04-05 07:00,Sea Floor to Summit 4
2021-04-04 07:00:00+00:00,04-04 07:00,Sea Floor to Summit 3
...,...,...
2020-11-19 04:45:00+00:00,11-19 04:45,Thursday Sweet Spot AM
2020-10-25 12:30:00+00:00,10-25 12:30,L’ENFER EN FRANCE
2020-10-24 15:00:00+00:00,10-24 15:00,Flemish For Beginners


## Personal Calendar for Signups

g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com

In [3]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = 'g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


In [4]:
df.index.max() 


Timestamp('2021-04-10 11:00:00+0000', tz='UTC')

## Read Events in Calendar which Are Within Timeframe of RGDB Data

Add one day to timeframe to compensate for unpredictable time zone handling.

Note IDs of existing events so we don't re-create them.

In [5]:
df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    df.loc[(df['Name'] == event.summary) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]

    #if event.start in df.loc[df['Name'] == event.summary].index:
    #    df.at[event.start, 'cal_id'] = event.event_id
    #    df.at[event.start, 'event_obj'] = event
    #    print('found ', event, event.timezone, event.event_id)



In [6]:
df.loc[df['found'] == False]

,Date,Name,cal_id,event_obj,found
Year_Date,,,,,
2021-04-07 04:00:00+00:00,04-07 04:00,The Recovery Ride,None,None,False
2021-04-03 10:00:00+00:00,04-03 10:00,RIAK E-DUATHLON: Race 3,None,None,False
2021-03-31 04:00:00+00:00,03-31 04:00,The Recovery Ride,None,None,False
2021-03-18 09:15:00+00:00,03-18 09:15,HUUB - Coalville Wheelers TT,None,None,False


In [7]:
for index, row in df.iterrows():

    if row['found'] == False:

        print('* Adding:', index, row['Name'])

        evntColor = '1'

        event = gcEvent(
            str(row['Name']),
            start=index,
            timezone='UTC',
            color = evntColor
        )
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else: 
        print('Skipping', index, row['Name'])

Skipping 2021-04-10 11:00:00+00:00 WKG's SDW. Stage 5
Skipping 2021-04-09 05:00:00+00:00 The Endurance Ride
* Adding: 2021-04-07 04:00:00+00:00 The Recovery Ride
ID after add: None Returned event ID: sh3k96c5uue28p8fqjdj1d7j78
Skipping 2021-04-05 07:00:00+00:00 Sea Floor to Summit 4
Skipping 2021-04-04 07:00:00+00:00 Sea Floor to Summit 3
Skipping 2021-04-03 11:00:00+00:00 WKG's SDW. Stage 4
* Adding: 2021-04-03 10:00:00+00:00 RIAK E-DUATHLON: Race 3
ID after add: None Returned event ID: 086o9jru15sh7a00gp1ful3c2k
Skipping 2021-04-03 07:00:00+00:00 Sea Floor to Summit 2
Skipping 2021-04-02 07:00:00+00:00 Sea Floor to Summit 1
Skipping 2021-04-02 05:00:00+00:00 The Endurance Ride
* Adding: 2021-03-31 04:00:00+00:00 The Recovery Ride
ID after add: None Returned event ID: mirlofupb92t2pqchl8p16h4cs
Skipping 2021-03-28 14:00:00+00:00 Lou's Sunday Group Ride
Skipping 2021-03-28 13:00:00+00:00 MRETT18 Lou's Team DST
Skipping 2021-03-27 14:00:00+00:00 Lou's Saturday Group Ride
Skipping 2021-0